In [14]:
import json
import os
import numpy as np
import pandas as pd

root = "verified"

identifier = []
photos = []
captions = []
collection_bools = []
collections = 0
photo_dump = 0
num_dump = 0
no_image = 0
no_caption = 0
has_quotes = 0
scraper_failed = 0
posts_per_user = []
for fn in os.listdir("profiles_" + root + "/"):
    if fn.endswith(".json"):
        f = open("profiles_" + root + "/" + fn)
        num_valid_entries = 0
        num_invalid_entries = 0
        data = json.load(f)
        try:
            posts = data["posts"]
        except:
            if data["num_of_posts"] > 0:
                scraper_failed += data["num_of_posts"]
            exit
        for p in posts:
            if len(p["imgs"]) > 1:
                collections += 1
                try:
                    num_valid_entries += 1
                    identifier.append(data["username"])
                    photos.append(p["imgs"][0])
                    if '"' in p["caption"]:
                        has_quotes += 1
                    captions.append(p["caption"].replace('"', '\''))
                    collection_bools.append(True)
                except:
                    print("collection failed")
            elif len(p["imgs"]) == 0:
                no_image += 1
                num_valid_entries += 1
            elif p["caption"] == "":
                no_caption += 1
                num_invalid_entries += 1
            else:
                num_valid_entries += 1
                if len(p["imgs"]) > 1:
                    print("collection")
                    collections += 1
                identifier.append(data["username"])
                photos.append(p["imgs"][0])
                if '"' in p["caption"]:
                    has_quotes += 1
                captions.append(p["caption"].replace('"', '\''))
                collection_bools.append(False)
        posts_per_user.append(num_valid_entries)
        scraper_failed += data["num_of_posts"] - num_valid_entries - num_invalid_entries
        f.close()

total = len(captions) + photo_dump + collections + no_image + no_caption
print("scraper failed", scraper_failed, round(scraper_failed * 100/ (total + scraper_failed), 2))
print("throwing out:", photo_dump + no_image + no_caption, round((photo_dump + no_image + no_caption) * 100 / total, 2))
print("\t no caption", no_caption, round((no_caption / total) * 100, 2))
print("\t no image", no_image, round((no_image / total) * 100, 2))
print("usable:", len(captions) + collections, round((len(captions)+ collections)*100/total, 2))
print("\t valid collections:", collections, round(collections* 100 / total, 2))
print("\t valid single photos:", len(captions), round(len(captions)*100/total, 2))
print("post per user:", posts_per_user)

with open('data_' + root + '.csv', 'w') as f:
    f.write("username,photo,caption,collection\n")
    for i in range(len(photos)):
        f.write("%s,%s,\"%s\",%s\n" % (identifier[i],photos[i], captions[i], str(collection_bools[i])))
f.close()

try:
    csv = pd.read_csv('data_' + root + '.csv', header=0)
    csv.drop_duplicates(keep="last",inplace=True)
    csv.to_csv('data_' + root + '.csv', index=False)
except:
    print('pandas stuff did not work')


scraper failed 31060 94.65
throwing out: 364 20.73
	 no caption 52 2.96
	 no image 312 17.77
usable: 1392 79.27
	 valid collections: 187 10.65
	 valid single photos: 1205 68.62
post per user: [84, 99, 97, 0, 79, 96, 48, 98, 100, 99, 98, 92, 96, 92, 12, 12, 91, 18, 95, 100, 11]
